In [26]:
import os
import os.path as osp
import numpy as np
import pandas as pd
import argparse

from ukge.datasets import KGTripleDataset
from ukge.models import TransE, DistMult, ComplEx, RotatE
from ukge.losses import compute_det_transe_distmult_loss, compute_det_complex_loss, compute_det_rotate_loss
from ukge.metrics import Evaluator

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
# from torch.utils.tensorboard import SummaryWriter

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

model_map = {
    'transe': TransE,
    'distmult': DistMult,
    'complex': ComplEx,
    'rotate': RotatE
}

loss_map = {
    'transe': compute_det_transe_distmult_loss,
    'distmult': compute_det_transe_distmult_loss,
    'complex': compute_det_complex_loss,
    'rotate': compute_det_rotate_loss
}

model = 'distmult'
dataset = 'cn15k'
confidence_score_function = 'logi'
hidden_dim = 128
num_neg_per_positive = 10
batch_size = 1024
lr = 0.01
weight_decay = 0.0005

model_checkpoint = '/home/mou/Projects/UKGE-FL/results/unc_cn15k_distmult/lr_0.01_hidden_dim_128_confi_logi/best_model_ndcg_exp.pth'

In [27]:
train_dataset = KGTripleDataset(dataset=dataset, split='train', num_neg_per_positive=num_neg_per_positive, deterministic=False)
val_dataset = KGTripleDataset(dataset=dataset, split='val')
test_dataset = KGTripleDataset(dataset=dataset, split='test')
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model_map[model](num_nodes=train_dataset.num_cons(), num_relations=train_dataset.num_rels(), hidden_channels=hidden_dim, confidence_score_function=confidence_score_function)

model.load_state_dict(torch.load(model_checkpoint)['state_dict'])
model = model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.999), lr=lr, weight_decay=weight_decay)


test_evaluator = Evaluator(test_dataloader, model, batch_size=batch_size, device=device)


In [28]:
pos_hrt, pos_target, neg_hn_rt, neg_hr_tn = next(iter(train_dataloader))
pos_hrt, pos_target, neg_hn_rt, neg_hr_tn = pos_hrt.long(), pos_target.float(), neg_hn_rt.long(), neg_hr_tn.long()
pos_hrt, pos_target, neg_hn_rt, neg_hr_tn = pos_hrt.to(device), pos_target.to(device), neg_hn_rt.to(device), neg_hr_tn.to(device) 
neg_hn_rt = neg_hn_rt.view(-1, neg_hn_rt.size(-1))
neg_hr_tn = neg_hr_tn.view(-1, neg_hr_tn.size(-1))

optimizer.zero_grad()
pred_pos_score = model.get_confidence_score(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])
pred_neg_hn_score = model.get_confidence_score(neg_hn_rt[:, 0], neg_hn_rt[:, 1], neg_hn_rt[:, 2])
pred_neg_tn_score = model.get_confidence_score(neg_hr_tn[:, 0], neg_hr_tn[:, 1], neg_hr_tn[:, 2])
neg_target = torch.zeros_like(pred_neg_hn_score)

In [29]:
print('pos_target:', pos_target)
print('pred_pos_score:', pred_pos_score)
print('pred_neg_hn_score:', pred_neg_hn_score)
print('pred_neg_tn_score:', pred_neg_tn_score)

pos_target: tensor([0.2754, 0.7093, 0.2341,  ..., 0.7093, 0.7093, 0.7093], device='cuda:0')
pred_pos_score: tensor([0.3150, 0.3150, 0.3150,  ..., 0.3150, 0.3150, 0.3150], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
pred_neg_hn_score: tensor([0.3150, 0.3150, 0.3150,  ..., 0.3150, 0.3150, 0.3150], device='cuda:0',
       grad_fn=<SigmoidBackward0>)
pred_neg_tn_score: tensor([0.3150, 0.3150, 0.3150,  ..., 0.3150, 0.3150, 0.3150], device='cuda:0',
       grad_fn=<SigmoidBackward0>)


In [30]:
pos_loss = criterion(pred_pos_score, pos_target)
neg_loss = (criterion(pred_neg_hn_score, neg_target) + criterion(pred_neg_tn_score, neg_target)) / 2
loss = pos_loss + neg_loss
loss.backward()
optimizer.step()

In [31]:

pred_pos_score = model.get_confidence_score(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])
pred_neg_hn_score = model.get_confidence_score(neg_hn_rt[:, 0], neg_hn_rt[:, 1], neg_hn_rt[:, 2])
pred_neg_tn_score = model.get_confidence_score(neg_hr_tn[:, 0], neg_hr_tn[:, 1], neg_hr_tn[:, 2])

In [32]:
print('pos_target:', pos_target)
print('pred_pos_score:', pred_pos_score)

pos_target: tensor([0.2754, 0.7093, 0.2341,  ..., 0.7093, 0.7093, 0.7093], device='cuda:0')
pred_pos_score: tensor([0.3172, 0.3172, 0.3172,  ..., 0.3172, 0.3172, 0.3172], device='cuda:0',
       grad_fn=<SigmoidBackward0>)


In [33]:
model_state_dict = model.state_dict()
for key in model_state_dict.keys():
    print(key)
    print(model_state_dict[key])

weights
tensor([-5.6144e-08], device='cuda:0')
bias
tensor([-0.7667], device='cuda:0')
node_emb.weight
tensor([[-3.4082e-32,  5.9898e-32, -7.3074e-33,  ...,  3.7819e-33,
          6.3872e-33, -6.2951e-32],
        [ 5.1940e-34, -5.5328e-28, -7.5746e-33,  ..., -8.0890e-32,
         -1.7348e-32,  7.0222e-32],
        [-2.2738e-32, -7.1393e-32,  3.3184e-32,  ..., -7.0916e-32,
          1.0592e-32,  3.5184e-32],
        ...,
        [-6.2515e-32, -5.4535e-32, -5.6645e-32,  ..., -6.1746e-32,
          5.7382e-32, -4.2419e-32],
        [ 2.8476e-32, -1.1275e-28,  8.2830e-32,  ...,  1.6614e-33,
          8.8556e-32, -1.0183e-31],
        [-4.8744e-32,  5.9291e-32,  6.0774e-32,  ...,  5.3987e-32,
         -5.8261e-32,  6.6685e-32]], device='cuda:0')
rel_emb.weight
tensor([[-7.9749e-20,  9.0417e-16, -4.9120e-16,  ...,  4.1401e-16,
          3.1908e-15,  1.6092e-14],
        [ 6.1925e-31,  1.0016e-20, -1.9920e-29,  ..., -5.1944e-27,
         -1.2896e-31, -6.9439e-16],
        [-1.2168e-22, -2.02

In [34]:
model.eval()
test_evaluator.update_hr_scores_map()
test_mean_ndcg = test_evaluator.get_mean_ndcg()
test_mse = test_evaluator.get_mse()
test_mae = test_evaluator.get_mae()

Updating hr_scores_map...


100%|██████████| 8063/8063 [01:25<00:00, 94.38it/s] 


In [35]:
print(test_mse)
print(test_mae)
print(test_mean_ndcg[0])
print(test_mean_ndcg[1])

0.25829725123496183
0.5873160022511443
0.1352762755276738
0.13376590751617082


In [36]:
hrtw_map = test_evaluator.hrtw_map
hr_scores_map = test_evaluator.hr_scores_map

In [37]:
hrt = [[h, r, t, hrtw_map[h][r][t], hr_scores_map[h][r][t]] for h in hrtw_map.keys() for r in hrtw_map[h].keys() for t in hrtw_map[h][r].keys()]
hrt_df = pd.DataFrame(hrt, columns=['head', 'tail', 'relation', 'score', 'prediction'])

In [38]:
hrt_df.to_csv('hrt_df.csv', index=False)

In [39]:
pos_hrt, pos_target = next(iter(test_dataloader))
pos_hrt, pos_target = pos_hrt.long(), pos_target.float()
pos_hrt, pos_target = pos_hrt.to(device), pos_target.to(device)
head_emb, rel_emb, tail_emb = model.get_embeddings(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])

In [40]:
print(head_emb.shape, rel_emb.shape, tail_emb.shape)
print(head_emb, rel_emb, tail_emb)

torch.Size([1024, 128]) torch.Size([1024, 128]) torch.Size([1024, 128])
tensor([[ 3.3727e-32,  1.9407e-23, -7.2701e-32,  ..., -7.2631e-33,
         -3.0053e-29, -7.4555e-32],
        [ 8.4886e-34,  1.0690e-23,  3.9289e-33,  ...,  8.0028e-32,
          4.0820e-32,  4.1573e-32],
        [ 5.8744e-32, -1.8589e-32, -1.2300e-32,  ..., -5.0476e-32,
         -7.2368e-32,  8.9714e-32],
        ...,
        [-4.5248e-32, -7.4693e-32, -5.5655e-32,  ...,  7.3185e-31,
          1.1890e-30,  4.8946e-30],
        [-5.8880e-32,  9.3092e-31,  4.9502e-32,  ..., -4.6989e-31,
         -9.4953e-33, -1.7723e-31],
        [ 5.3846e-32, -4.6955e-32,  5.9569e-32,  ..., -4.6016e-32,
         -5.8411e-32, -1.7823e-32]], device='cuda:0',
       grad_fn=<EmbeddingBackward0>) tensor([[-7.9749e-20,  9.0417e-16, -4.9120e-16,  ...,  4.1401e-16,
          3.1908e-15,  1.6092e-14],
        [-7.9749e-20,  9.0417e-16, -4.9120e-16,  ...,  4.1401e-16,
          3.1908e-15,  1.6092e-14],
        [-7.9749e-20,  9.0417e-16, -

In [41]:
plau_score = model.get_plausibility_score(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])
print(plau_score)

tensor([-4.5654e-42,  9.8091e-45,  0.0000e+00,  ...,  0.0000e+00,
        -8.7946e-22,  1.0344e-40], device='cuda:0', grad_fn=<SumBackward1>)


In [42]:
conf_score = model.get_confidence_score(pos_hrt[:, 0], pos_hrt[:, 1], pos_hrt[:, 2])
print(conf_score)

tensor([0.3172, 0.3172, 0.3172,  ..., 0.3172, 0.3172, 0.3172], device='cuda:0',
       grad_fn=<SigmoidBackward0>)


In [43]:
print(model.fc_in, model.fc_out)

Parameter containing:
tensor([-5.6144e-08], device='cuda:0', requires_grad=True) Parameter containing:
tensor([-0.7667], device='cuda:0', requires_grad=True)
